# Setup
 installation of PyG on Colab

In [ ]:
import torch
import os
print("PyTorch has version {}".format(torch.__version__))
!python --version
!nvidia-smi

PyTorch has version 2.6.0+cu124
Python 3.11.13
Sun Jun  8 19:30:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+

Make sure the version of torch matches cuda, torch sparse and torch vision. More information can be found on the [PyG's installation page](https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html).

In [ ]:
# Install torch geometric

# !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
# !wget https://pytorch-geometric.com/whl/torch-1.13.1+cu116/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
# !pip install torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
# !wget https://pytorch-geometric.com/whl/torch-1.13.1+cu116/torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl
# !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html

# !wget torch_scatter-2.1.2+pt26cu124-cp311-cp311-linux_x86_64.whl
# # !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
# !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.6.0+cu124.html
# !wget torch_sparse-0.6.18+pt26cu124-cp311-cp311-linux_x86_64.whl
# !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.6.0+cu124.html
# !pip install ogb --upgrade

Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
--2025-06-08 19:32:22--  https://pytorch-geometric.com/whl/torch-1.13.1+cu116/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
Resolving pytorch-geometric.com (pytorch-geometric.com)... 3.165.75.113, 3.165.75.26, 3.165.75.31, ...
Connecting to pytorch-geometric.com (pytorch-geometric.com)|3.165.75.113|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2025-06-08 19:32:23 ERROR 403: Forbidden.

ERROR: torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl is not a supported wheel on this platform.
--2025-06-08 19:32:24--  https://pytorch-geometric.com/whl/torch-1.13.1+cu116/torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl
Resolving pytorch-geometric.com (pytorch-geometric.com)... 3.165.75.113, 3.165.75.26, 3.165.75.31, ...
Connecting to pytorch-geometric.com (pytorch-geometric.com)|3.165.75.113|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2025-06-08 19:32:25 ERROR 403: Fo

In [ ]:
# 1) scatter & sparse (you already have these, but for completeness):
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
!pip install torch-sparse  -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

# 2) cluster & spline:
!pip install torch-cluster     -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

# 3) finally, the meta-package
!pip install torch-geometric   -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

!pip install ogb --upgrade

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 75.7 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.4 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.4 MB/s eta 0:00:00


# 1) PyTorch Geometric (Datasets and Data)


PyTorch Geometric has two classes for storing and/or transforming graphs into tensor format. One is `torch_geometric.datasets`, which contains a variety of common graph datasets. Another is `torch_geometric.data`, which provides the data handling of graphs in PyTorch tensors.

In this section, we will learn how to use `torch_geometric.datasets` and `torch_geometric.data` together.

## PyG Datasets

The `torch_geometric.datasets` class has many common graph datasets. Here we will explore its usage through one example dataset.

In [ ]:
import torch_geometric.transforms as T
import torch_geometric
from torch_geometric.data.data import DataTensorAttr, DataEdgeAttr
from torch_geometric.data.storage import GlobalStorage
from ogb.nodeproppred import PygNodePropPredDataset

import torch.serialization
import os
torch.serialization.add_safe_globals([DataEdgeAttr, DataTensorAttr, GlobalStorage])
# TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD=1
# torch.serialization.add_safe_globals({'DataEdgeAttr': DataEdgeAttr})

dataset_name = 'ogbn-arxiv'
# dataset = PygNodePropPredDataset(name=dataset_name, transform=T.ToSparseTensor())
# with torch.serialization.safe_globals([torch_geometric.data.data.DataEdgeAttr]) as context:
dataset = PygNodePropPredDataset(name=dataset_name)
print('The {} dataset has {} graph'.format(dataset_name, len(dataset)))
data = dataset[0]
print(data)

# dataset_name = 'ogbn-arxiv'
# dataset = PygNodePropPredDataset(name=dataset_name, transform=T.ToSparseTensor())
# data = torch.load(dataset.processed_paths[0], weights_only=False)
# print(data)

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:08<00:00,  9.78it/s]


Extracting dataset/arxiv.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 3466.37it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 2490.68it/s]

Saving...


The ogbn-arxiv dataset has 1 graph
Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343, 1], y=[169343, 1])


Done!


In [ ]:
from torch_geometric.datasets import TUDataset
root = './enzymes'
name = 'ENZYMES'

# The ENZYMES dataset
pyg_dataset= TUDataset(root, name)

# You will find that there are 600 graphs in this dataset
print(pyg_dataset)

Processing...


ENZYMES(600)


Done!


In [ ]:
# What is the number of classes and number of features in the dataset
def get_num_classes(pyg_dataset):
  num_classes = 0
  num_classes = pyg_dataset.num_classes
  return num_classes

def get_num_features(pyg_dataset):
  num_features = 0
  num_classes = pyg_dataset.num_features


  return num_features

num_classes = get_num_classes(pyg_dataset)
num_features = get_num_features(pyg_dataset)
print("{} dataset has {} classes".format(name, num_classes))
print("{} dataset has {} features".format(name, num_features))

ENZYMES dataset has 6 classes
ENZYMES dataset has 0 features


## PyG Data

Each PyG dataset stores a list of `torch_geometric.data.Data` objects, where each `torch_geometric.data.Data` object represents a graph. We can easily get the `Data` object by indexing into the dataset.

For more information such as what is stored in the `Data` object, please refer to the [documentation](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data).

In [ ]:
# What is the number of classes and number of features in the
num_classes = get_num_classes(pyg_dataset)
num_features = get_num_features(pyg_dataset)
print("{} dataset has {} classes".format(name, num_classes))
print("{} dataset has {} features".format(name, num_features))

# How many edges does the graph with index 200 have?
pyg_dataset[200].num_edges
pyg_dataset[200].contains_isolated_nodes() # False

ENZYMES dataset has 6 classes
ENZYMES dataset has 0 features


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  warnings.warn(out)


False

In [ ]:
def get_graph_num_edges(pyg_dataset, idx):

  num_edges = 0


  edges = []
  for edge in pyg_dataset[200].to_dict()['edge_index'].T:
    if (list(edge) not in edges) & ([edge[1], edge[0]] not in edges):
      edges.append(list(edge))
  num_edges = len(edges)


  return num_edges

# find out the edges connected to one node
idx = 200
num_edges = get_graph_num_edges(pyg_dataset, idx)
print('Graph with index {} has {} edges'.format(idx, num_edges))

Graph with index 200 has 53 edges


# 2) Open Graph Benchmark (OGB)

The Open Graph Benchmark (OGB) is a collection of realistic, large-scale, and diverse benchmark datasets for machine learning on graphs. Its datasets are automatically downloaded, processed, and split using the OGB Data Loader. The model performance can then be evaluated by using the OGB Evaluator in a unified manner.

## Question 4: How many features are in the ogbn-arxiv graph? (5 points)

In [ ]:
def graph_num_features(data):

  num_features = 0
  num_features = data.num_features

  return num_features

# How many features are in the ogbn-arxiv graph?
num_features = graph_num_features(data)
print('The graph has {} features'.format(num_features))

The graph has 128 features


In [ ]:
data.num_edge_features, data.num_node_features

(0, 128)

# 3) GNN: Node Property Prediction

In this section we will build our first graph neural network using PyTorch Geometric. Then we will apply it to the task of node property prediction (node classification).

Specifically, we will use GCN as the foundation for your graph neural network ([Kipf et al. (2017)](https://arxiv.org/pdf/1609.02907.pdf)). To do so, we will work with PyG's built-in `GCNConv` layer.

## Setup

In [ ]:
import torch
import pandas as pd
import torch.nn.functional as F
print(torch.__version__)

# The PyG built-in GCNConv
from torch_geometric.nn import GCNConv

import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

2.6.0+cu124


In [ ]:
data

Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343, 1], y=[169343, 1])

## Load and Preprocess the Dataset

In [ ]:
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name=dataset_name,
                                transform=T.ToSparseTensor())
data = dataset[0]

# Make the adjacency matrix to symmetric
# data.adj_t = data.adj_t.to_symmetric()
data.adj_t = data.adj_t + data.adj_t.t()
# data.adj_t = data.adj_t.setdiag()  # Optionally set diagonal elements
# Optionally set diagonal elements
# 1) Make symmetric:
data.adj_t = data.adj_t.to_symmetric()

# 2) Zero out the diagonal:
data.adj_t = data.adj_t.set_diag(0.)



device = 'cuda' if torch.cuda.is_available() else 'cpu'

# If you use GPU, the device should be cuda
print('Device: {}'.format(device))

data = data.to(device)
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

Device: cuda


In [ ]:
data.adj_t

SparseTensor(row=tensor([     0,      0,      0,  ..., 169342, 169342, 169342], device='cuda:0'),
             col=tensor([     0,    411,    640,  ...,  27824, 158981, 169342], device='cuda:0'),
             size=(169343, 169343), nnz=2484941, density=0.01%)

In [ ]:
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

## GCN Model

Now we will implement our GCN model!

Please follow the figure below to implement the `forward` function.


![test](https://drive.google.com/uc?id=128AuYAXNXGg7PIhJJ7e420DoPWKb-RtL)

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):

        super(GCN, self).__init__()

        # A list of GCNConv layers
        self.convs = None

        # A list of 1D batch normalization layers
        self.bns = None

        # The log softmax layer
        self.softmax = None
        self.convs = torch.nn.ModuleList(
            [GCNConv(in_channels = input_dim, out_channels=hidden_dim)] +
            [GCNConv(in_channels = hidden_dim, out_channels=hidden_dim) for i in range(num_layers - 2)] +
            [GCNConv(in_channels = hidden_dim, out_channels=output_dim)]
            )
        self.bns = torch.nn.ModuleList(
            [torch.nn.BatchNorm1d(num_features=hidden_dim) for i in range(num_layers - 1)]
            )
        self.softmax = torch.nn.LogSoftmax(-1)

        # Probability of an element getting zeroed
        self.dropout = dropout

        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):

        out = None


        for conv, bn in zip(self.convs[:-1], self.bns):
          x = F.relu(bn(conv(x, adj_t)))
          if self.training:
            x = F.dropout(x, self.dropout)
        # layers = [torch.nn.Sequential(conv(x, adj_t), bn, F.relu, F.dropout(input = x, p =self.dropout)) if self.training else torch.nn.Sequential(conv, bn, F.relu) for conv, bn in zip(self.convs[:-1], self.bns)]
        # x = layers(x)

        # final layer
        x = self.convs[-1](x, adj_t)
        out = x if self.return_embeds else self.softmax(x)

        return out

In [ ]:
def train(model, data, train_idx, optimizer, loss_fn):

    model.train()
    loss = 0


    optimizer.zero_grad()
    output = model(x = data.x, adj_t = data.adj_t) # Pass params to forward()
    y_hat = output[train_idx]
    y = data.y[train_idx]
    loss = loss_fn(y_hat, y.reshape(-1))

    loss.backward()
    optimizer.step()

    return loss.item()

In [ ]:
# Test function here
@torch.no_grad()
def test(model, data, split_idx, evaluator, save_model_results=False):
    # A function that tests the model by
    # using the given split_idx and evaluator.
    model.eval()

    # The output of model on all data
    out = None
    out = model(data.x, data.adj_t)


    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    if save_model_results:
      print ("Saving Model Predictions")

      data = {}
      data['y_pred'] = y_pred.view(-1).cpu().detach().numpy()

      df = pd.DataFrame(data=data)
      # Save locally as csv
      df.to_csv('ogbn-arxiv_node.csv', sep=',', index=False)


    return train_acc, valid_acc, test_acc

In [ ]:
args = {
      'device': device,
      'num_layers': 3,
      'hidden_dim': 256,
      'dropout': 0.5,
      'lr': 0.01,
      'epochs': 100,
}


In [ ]:
model = GCN(data.num_features, args['hidden_dim'],
            dataset.num_classes, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbn-arxiv')

In [ ]:
# Please do not change these args
# Training should take <10min using GPU runtime
import copy
# reset the parameters to initial random value
model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = F.nll_loss

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, optimizer, loss_fn)
  result = test(model, data, split_idx, evaluator)
  train_acc, valid_acc, test_acc = result
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

Epoch: 01, Loss: 4.2215, Train: 26.66%, Valid: 29.61% Test: 26.51%
Epoch: 02, Loss: 2.3387, Train: 26.63%, Valid: 29.45% Test: 33.86%
Epoch: 03, Loss: 1.9344, Train: 21.62%, Valid: 12.92% Test: 11.59%
Epoch: 04, Loss: 1.7622, Train: 24.81%, Valid: 15.84% Test: 13.74%
Epoch: 05, Loss: 1.6569, Train: 28.97%, Valid: 18.65% Test: 16.83%
Epoch: 06, Loss: 1.5740, Train: 33.86%, Valid: 25.73% Test: 25.55%
Epoch: 07, Loss: 1.5027, Train: 38.96%, Valid: 31.66% Test: 33.14%
Epoch: 08, Loss: 1.4590, Train: 40.47%, Valid: 31.93% Test: 32.58%
Epoch: 09, Loss: 1.4089, Train: 40.38%, Valid: 31.99% Test: 32.52%
Epoch: 10, Loss: 1.3701, Train: 40.93%, Valid: 33.07% Test: 33.93%
Epoch: 11, Loss: 1.3339, Train: 42.48%, Valid: 36.54% Test: 38.04%
Epoch: 12, Loss: 1.3099, Train: 44.98%, Valid: 41.09% Test: 43.17%
Epoch: 13, Loss: 1.2859, Train: 48.08%, Valid: 45.78% Test: 48.34%
Epoch: 14, Loss: 1.2668, Train: 51.29%, Valid: 50.37% Test: 53.53%
Epoch: 15, Loss: 1.2459, Train: 53.72%, Valid: 54.00% Test: 56

In [ ]:
import os
os.makedirs('models', exist_ok=True)
torch.save(best_model.state_dict(), 'models/best_model.pth')
best_model.load_state_dict(torch.load('models/best_model.pth'))
best_model.eval()  # Set the model to evaluation mode

GCN(
  (convs): ModuleList(
    (0): GCNConv(128, 256)
    (1): GCNConv(256, 256)
    (2): GCNConv(256, 40)
  )
  (bns): ModuleList(
    (0-1): 2 x BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (softmax): LogSoftmax(dim=-1)
)

In [ ]:
best_result = test(best_model, data, split_idx, evaluator, save_model_results=True)
train_acc, valid_acc, test_acc = best_result
print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')

Saving Model Predictions
Best model: Train: 73.22%, Valid: 71.91% Test: 71.23%


### Bonus: GNNExaplainer

Author: Tommy Xie (TommyXie_@outlook.com)

In [ ]:
from torch_geometric.utils import to_edge_index

# GNNExplainer in PyTorch Geometric (PyG) currently does not support inputs in the SparseTensor
# Convert SparseTensor to edge_index and edge_attr
edge_index, edge_attr = to_edge_index(data.adj_t)
# Update the data object
data.edge_index = edge_index
data.edge_attr = edge_attr
del edge_attr, edge_index

In [ ]:
from torch_geometric.explain import Explainer, GNNExplainer
os.makedirs('figs_tabs', exist_ok=True)

explainer = Explainer(
    model=best_model,
    algorithm=GNNExplainer(epochs=50),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='log_probs',
    ),
)
node_index = 10


explanation = explainer(data.x, data.edge_index, index=node_index)

print(f'Generated explanations in {explanation.available_explanations}')

path = 'figs_tabs/feature_importance.png'
explanation.visualize_feature_importance(path, top_k=10)
print(f"Feature importance plot has been saved to '{path}'")
print(f'Generated explanations in {explanation.available_explanations}')

Generated explanations in ['node_mask', 'edge_mask']
Feature importance plot has been saved to 'figs_tabs/feature_importance.png'
Generated explanations in ['node_mask', 'edge_mask']


In [ ]:
# # perform explantion on a subgraph instead
# from torch_geometric.utils import k_hop_subgraph
# from torch_geometric.data import Data

# # Parameters
# node_idx = 10  # Node of interest
# num_hops = 5   # Number of hops to include in the subgraph

# subset, edge_index, mapping, edge_mask = k_hop_subgraph(
#     node_idx=node_idx,
#     num_hops=num_hops,
#     edge_index=edge_index,
#     relabel_nodes=True
# )

# # Create the subgraph data object
# sub_data = Data(
#     x=data.x[subset],
#     edge_index=edge_index
# )

# # Apply GNNExplainer on the subgraph
# explanation = explainer(sub_data.x, sub_data.edge_index, index=mapping)


# 4) GNN: Graph Property Prediction

In this section we will create a graph neural network for graph property prediction (graph classification).


## Load and preprocess the dataset

In [ ]:
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.data import DataLoader
from tqdm.notebook import tqdm

# Load the dataset
dataset = PygGraphPropPredDataset(name='ogbg-molhiv')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))

split_idx = dataset.get_idx_split()

# Check task type
print('Task type: {}'.format(dataset.task_type))

Downloaded 0.00 GB: 100%|██████████| 3/3 [00:02<00:00,  1.11it/s]
Processing...


Extracting dataset/hiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 41127/41127 [00:00<00:00, 107479.72it/s]


Converting graphs into PyG objects...


100%|██████████| 41127/41127 [00:01<00:00, 29484.22it/s]


Saving...
Device: cuda
Task type: binary classification


Done!


In [ ]:
# Load the dataset splits into corresponding dataloaders
# We will train the graph classification task on a batch of 32 graphs
# Shuffle the order of graphs for training set
train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False, num_workers=0)

/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
# Please do not change the args
args = {
    'device': device,
    'num_layers': 5,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.001,
    'epochs': 30,
}
args

{'device': 'cuda',
 'num_layers': 5,
 'hidden_dim': 256,
 'dropout': 0.5,
 'lr': 0.001,
 'epochs': 30}

## Graph Prediction Model

### Graph Mini-Batching
Before diving into the actual model, we introduce the concept of mini-batching with graphs. In order to parallelize the processing of a mini-batch of graphs, PyG combines the graphs into a single disconnected graph data object (*torch_geometric.data.Batch*). *torch_geometric.data.Batch* inherits from *torch_geometric.data.Data* (introduced earlier) and contains an additional attribute called `batch`.

The `batch` attribute is a vector mapping each node to the index of its corresponding graph within the mini-batch:

    batch = [0, ..., 0, 1, ..., n - 2, n - 1, ..., n - 1]

This attribute is crucial for associating which graph each node belongs to and can be used to e.g. average the node embeddings for each graph individually to compute graph level embeddings.



### Implemention
Now, we have all of the tools to implement a GCN Graph Prediction model!  

We will reuse the existing GCN model to generate `node_embeddings` and then use  `Global Pooling` over the nodes to create graph level embeddings that can be used to predict properties for the each graph. Remeber that the `batch` attribute will be essential for performining Global Pooling over our mini-batch of graphs.

In [ ]:
from ogb.graphproppred.mol_encoder import AtomEncoder
from torch_geometric.nn import global_add_pool, global_mean_pool

### GCN to predict graph property
class GCN_Graph(torch.nn.Module):
    def __init__(self, hidden_dim, output_dim, num_layers, dropout):
        super(GCN_Graph, self).__init__()

        # Load encoders for Atoms in molecule graphs
        self.node_encoder = AtomEncoder(hidden_dim)

        # Node embedding model
        # Note that the input_dim and output_dim are set to hidden_dim
        self.gnn_node = GCN(hidden_dim, hidden_dim,
            hidden_dim, num_layers, dropout, return_embeds=True)

        self.pool = None


        self.pool = global_mean_pool

        # Output layer
        self.linear = torch.nn.Linear(hidden_dim, output_dim)


    def reset_parameters(self):
      self.gnn_node.reset_parameters()
      self.linear.reset_parameters()

    def forward(self, batched_data):

        # Extract important attributes of our mini-batch
        x, edge_index, batch = batched_data.x, batched_data.edge_index, batched_data.batch
        embed = self.node_encoder(x)

        out = None
        node_embed = self.gnn_node(embed, edge_index)
        features = self.pool(node_embed, batch)
        out = self.linear(features)

        return out

In [ ]:
def train(model, device, data_loader, optimizer, loss_fn):
    # a function that trains your model by
    # using the given optimizer and loss_fn.
    model.train()
    loss = 0

    for step, batch in enumerate(tqdm(data_loader, desc="Iteration")):
      batch = batch.to(device)

      if batch.x.shape[0] == 1 or batch.batch[-1] == 0:
          pass
      else:
        ## ignore nan targets (unlabeled) when computing training loss.
        is_labeled = batch.y == batch.y
        optimizer.zero_grad()
        y_hat = model(batch)[is_labeled]
        y = batch.y[is_labeled]
        loss = loss_fn(y_hat, y.float())
        loss.backward()
        optimizer.step()

    return loss.item()

In [ ]:
# The evaluation function
def eval(model, device, loader, evaluator, save_model_results=False, save_file=None):
    model.eval()
    y_true = []
    y_pred = []

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)

        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                pred = model(batch)

            y_true.append(batch.y.view(pred.shape).detach().cpu())
            y_pred.append(pred.detach().cpu())

    y_true = torch.cat(y_true, dim = 0).numpy()
    y_pred = torch.cat(y_pred, dim = 0).numpy()

    input_dict = {"y_true": y_true, "y_pred": y_pred}

    if save_model_results:
        print ("Saving Model Predictions")

        # Create a pandas dataframe with a two columns
        # y_pred | y_true
        data = {}
        data['y_pred'] = y_pred.reshape(-1)
        data['y_true'] = y_true.reshape(-1)

        df = pd.DataFrame(data=data)
        # Save to csv
        df.to_csv('ogbg-molhiv_graph_' + save_file + '.csv', sep=',', index=False)

    return evaluator.eval(input_dict)

In [ ]:
model = GCN_Graph(args['hidden_dim'],
            dataset.num_tasks, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbg-molhiv')

In [ ]:
# Please do not change these args
# Training should take <10min using GPU runtime
import copy

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = torch.nn.BCEWithLogitsLoss()

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  print('Training...')
  loss = train(model, device, train_loader, optimizer, loss_fn)

  print('Evaluating...')
  train_result = eval(model, device, train_loader, evaluator)
  val_result = eval(model, device, valid_loader, evaluator)
  test_result = eval(model, device, test_loader, evaluator)

  train_acc, valid_acc, test_acc = train_result[dataset.eval_metric], val_result[dataset.eval_metric], test_result[dataset.eval_metric]
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

Training...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Evaluating...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

...

Evaluating...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Epoch: 30, Loss: 0.0365, Train: 84.28%, Valid: 81.04% Test: 77.48%


In [ ]:

train_acc = eval(best_model, device, train_loader, evaluator)[dataset.eval_metric]
valid_acc = eval(best_model, device, valid_loader, evaluator, save_model_results=True, save_file="valid")[dataset.eval_metric]
test_acc  = eval(best_model, device, test_loader, evaluator, save_model_results=True, save_file="test")[dataset.eval_metric]

print(f'Best model: '
    f'Train: {100 * train_acc:.2f}%, '
    f'Valid: {100 * valid_acc:.2f}% '
    f'Test: {100 * test_acc:.2f}%')

Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Saving Model Predictions


Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Saving Model Predictions
Best model: Train: 84.28%, Valid: 81.04% Test: 77.48%


Experiment with the two other global pooling layers in Pytorch Geometric.